# Edge Search

Notebook to search for extreme heat events that overlap years.
Fix to resolve problem TBD.

By Cascade Tuholske, 2019-10-19


### Preliminary Findings

- In the entire record, there are 97 events that start on Jan 1.
- In the entire record, there are 94 events that end of Dec 31.

Of these, it looks like 5 were from the same city and bridged two years

#### Depdencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
fn = "/home/cascade/projects/data_out_urbanheat/All_data20191107.csv"  # Note: Need ?dl=1 to make sure this file gets read correctly
df = pd.read_csv(fn)
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Event_ID', 'ID_HDC_G0', 'CTR_MN_NM',
       'total_days', 'duration', 'avg_temp', 'avg_intensity', 'tot_intensity',
       'event_dates', 'intensity', 'tmax', 'year', 'region', 'sub-region',
       'intermediate-region', 'GCPNT_LAT', 'GCPNT_LON'],
      dtype='object')

#### Find Edges

Build query to find dates XXXX.12.31 and XXXX.01.01 
Events col are strings

In [5]:
df.dtypes

Unnamed: 0               int64
Unnamed: 0.1             int64
Event_ID                 int64
ID_HDC_G0                int64
CTR_MN_NM               object
total_days               int64
duration                 int64
avg_temp               float64
avg_intensity          float64
tot_intensity          float64
event_dates             object
intensity               object
tmax                    object
year                     int64
region                  object
sub-region              object
intermediate-region     object
GCPNT_LAT              float64
GCPNT_LON              float64
dtype: object

In [24]:
def date_search(df, date):

    event_id_list = []
    event_dates_list = []
    city_id_list = []
    event_year_list = []
    
    for index, row in df.iterrows():
        if date in row['event_dates']:
            
            event_id = row['Event_ID']
            event_dates = row['event_dates']
            city_id = row['ID_HDC_G0']
            event_year = row['year']
            
            event_id_list.append(event_id)
            event_dates_list.append(event_dates)
            city_id_list.append(city_id)
            event_year_list.append(event_year)
    
    df_out = pd.DataFrame()
    df_out['ID_HDC_G0'] = city_id_list
    df_out['Event_ID'] = event_id_list
    df_out['event_dates'] = event_dates_list
    df_out['year'] = event_year_list
    
    return df_out

In [25]:
# Dec 31 Events

df_1231 = date_search(df, '12.31')

In [26]:
# Jan 1 Events

df_0101 = date_search(df, '01.01')

In [27]:
df_1231.head()

,ID_HDC_G0,Event_ID,event_dates,year
0,1506,633720,['2008.12.31'],2008
1,5519,117936,['1987.12.29' '1987.12.30' '1987.12.31'],1987
2,5519,849342,['2015.12.31'],2015
3,5551,117949,['1987.12.30' '1987.12.31'],1987
4,5551,849363,['2015.12.31'],2015


In [37]:
print(len(df_0101))
print(len(df_1231))

94
97


In [33]:
# Merge based on cities

merge = df_0101.merge(df_1231, on = 'ID_HDC_G0', how = 'inner')

In [45]:
merge.head()

,ID_HDC_G0,Event_ID_x,event_dates_x,year_x,Event_ID_y,event_dates_y,year_y
0,5519,451678,['2001.01.20'],2001,117936,['1987.12.29' '1987.12.30' '1987.12.31'],1987
1,5519,451678,['2001.01.20'],2001,849342,['2015.12.31'],2015
2,5534,45819,['1984.01.01' '1984.01.02'],1984,90648,['1986.12.31'],1986
3,5534,45819,['1984.01.01' '1984.01.02'],1984,117974,['1987.12.28' '1987.12.29' '1987.12.30' '1987....,1987
4,5534,45819,['1984.01.01' '1984.01.02'],1984,327405,['1996.12.30' '1996.12.31'],1996


In [47]:
merge.dtypes

ID_HDC_G0         int64
Event_ID_x        int64
event_dates_x    object
year_x            int64
Event_ID_y        int64
event_dates_y    object
year_y            int64
dtype: object

In [40]:
#merge.to_csv('/home/cascade/projects/data_out_urbanheat/jandecoverlap.csv')

In [66]:
out = []

for i, year in merge.iterrows():
        if year['year_x'] - year['year_y'] == 1:
            out.append(i)


In [76]:
merge.loc[out]

,ID_HDC_G0,Event_ID_x,event_dates_x,year_x,Event_ID_y,event_dates_y,year_y
15,5534,141081,['1988.01.01' '1988.01.02'],1988,117974,['1987.12.28' '1987.12.29' '1987.12.30' '1987....,1987
29,5534,404842,['1999.01.01' '1999.01.02' '1999.01.03' '1999....,1999,376959,['1998.12.31'],1998
120,5534,882573,['2016.01.01' '2016.01.02' '2016.01.03'],2016,849422,['2015.12.31'],2015
129,5541,141093,['1988.01.01' '1988.01.02'],1988,117989,['1987.12.28' '1987.12.29' '1987.12.30' '1987....,1987
138,5541,404854,['1999.01.01' '1999.01.02' '1999.01.03' '1999....,1999,376974,['1998.12.31'],1998
183,5541,882595,['2016.01.01' '2016.01.02' '2016.01.03'],2016,849443,['2015.12.31'],2015
205,5520,141104,['1988.01.01' '1988.01.02'],1988,118003,['1987.12.28' '1987.12.29' '1987.12.30' '1987....,1987
236,5520,882617,['2016.01.01' '2016.01.02' '2016.01.03'],2016,849463,['2015.12.31'],2015
248,5390,404896,['1999.01.01'],1999,377007,['1998.12.31'],1998
273,5390,882679,['2016.01.01' '2016.01.02'],2016,849508,['2015.12.31'],2015


In [ ]:
dec31 = np.sort(df_1231['year'])
dec3

## NEXT:

I need to write a function that compares years with the resulting subset of Dec 31 and Jan 1 and then I think we will just drop all these from the record and then add them on as heat waves that overlap years later. There should be about 100 of them